In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd
import pickle

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:

train_data = pickle.load( open( "../train_data.p", "rb" ) )
train_labels = pickle.load( open( "../train_labels.p", "rb" ) )
test_data = pickle.load( open( "../test_data.p", "rb" ) )
test_labels = pickle.load( open( "../test_labels.p", "rb" ) )
val_data = pickle.load( open( "../test_data.p", "rb" ) )
val_labels = pickle.load( open( "../test_labels.p", "rb" ) )


In [107]:
#######################################################
####       this is for no-gene data prep           ####
#######################################################

data= pd.read_csv('../GenomeCRISPR_full05112017.csv')
toppub_all_data = data[data['pubmed'] ==29083409]
# groupby celline and sequence
toppub_grouped = toppub_all_data.groupby(['sequence','symbol']).median()['effect'].reset_index()
# groupby sequence to get unique seqs
groupby_seq = toppub_grouped.groupby(['sequence']).count().reset_index()
groupby_gene = toppub_grouped.groupby(['symbol']).count().reset_index()

# keeping only sequence column
unique_seq_toppub = groupby_seq['sequence']
groupby_gene = groupby_gene['symbol']

e_dict = {'A':[1,0,0,0], 'C':[0,1,0,0], 'G':[0,0,1,0], 'T':[0,0,0,1]}
def one_hot(x):
    return e_dict[x]

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
# onehot encode the unique seqs
one_hot_un_seq = unique_seq_toppub.apply(lambda x: map(one_hot, x))

In [109]:
# reset indices
unique_seq_toppub = unique_seq_toppub.reset_index()
one_hot_un_seq = one_hot_un_seq.reset_index()
groupby_gene = groupby_gene.reset_index()
# append onehot to seqs
unique_seq_toppub['one_hot'] = one_hot_un_seq['sequence']
# merge one hot encoding to data
toppub_one_hot = toppub_grouped.merge(unique_seq_toppub,left_on="sequence",right_on="sequence", how="left")

# one hot features without gene
features_no_gene = np.ndarray(shape=(toppub_one_hot.shape[0],23,4), dtype=int)
for i in range(toppub_one_hot.shape[0]):
    for j in range(len(toppub_one_hot['one_hot'][i])):
        features_no_gene[i][j] = toppub_one_hot['one_hot'][i][j]
        
pickle.dump(features_no_gene, open("../features_no_gene.p", "wb"))

NameError: name 'labels' is not defined

In [111]:
# regression labels
def convert_labels(y):
    return (y + 10)/float(20)
labels = toppub_one_hot['effect']
labels = np.array(list(map(lambda x: convert_labels(x), labels)))
# labels = np.eye(num_classes)[labels]

# split into training and testing no gene
msk = np.random.rand(len(features_no_gene)) < 0.8
train_data = features_no_gene[msk]
train_labels = labels[msk]
test_val_data = features_no_gene[~msk]
test_val_labels = labels[~msk]

msk = np.random.rand(len(test_val_data)) < 0.5
test_data = test_val_data[msk]
test_labels = test_val_labels[msk]
val_data = test_val_data[~msk]
val_labels = test_val_labels[~msk]

# for without gene
pickle.dump(train_data, open("../train_data2.p", "wb"))
pickle.dump(train_labels, open("../train_labels2.p", "wb"))
pickle.dump(test_data, open("../test_data2.p", "wb"))
pickle.dump(test_labels, open("../test_labels2.p", "wb"))
pickle.dump(val_data, open("../val_data2.p", "wb"))
pickle.dump(val_labels, open("../val_labels2.p", "wb"))


#######################################################
####            end of no-gene data prep           ####
#######################################################



In [112]:
# Training Parameters
learning_rate = 0.1
training_steps = 10000
batch_size = 1 # 128
display_step = 200

In [113]:
# Network Parameters
num_input = 4   # 28 # MNIST data input (img shape: 28*28)
timesteps = 23  # 28 # timesteps
num_hidden = 50 # 128 # hidden layer num of features
num_classes = 1 # 10 # MNIST total classes (0-9 digits)

In [114]:
#this resets the graph - re-run tensor-flow specific things after it
tf.reset_default_graph() 

In [115]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [116]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    #'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [117]:
def BiRNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)          #Unstack->list of 'timesteps' tensors of shape (batch_size, num_input)

    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)   # Forward direction cell
    outputs,_= rnn.static_rnn(lstm_fw_cell,x,dtype="float32")
    #lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)  # Backward direction cell
 
    #try:      # Get lstm cell output
    #    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    #except Exception: # Old TensorFlow version only returns outputs not states
    #    outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output then sigmoid result to get output in range [0,1]
    return tf.nn.sigmoid(tf.matmul(outputs[-1], weights['out']) + biases['out'])

In [118]:
prediction = BiRNN(X, weights, biases)

#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=Y))
loss_op = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y, predictions=prediction))
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  # switch to adam optimizer
train_op = optimizer.minimize(loss_op)  

# correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
# correct_pred = tf.equal(prediction, Y)
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
accuracy = tf.reduce_mean(tf.losses.mean_squared_error(Y, prediction))

# Initialize the variables 
init = tf.global_variables_initializer()

In [119]:
def get_train_data(batch_size, x, y):
    i = np.random.randint(0,x.shape[0], size=(batch_size))
    return np.array(x[i]), np.array(y[i])

In [120]:
#init=tf.global_variables_initializer()                  #initialize variables
x, y = train_data, train_labels
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<10000:
        batch_x, batch_y = get_train_data(batch_size, x,y)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        batch_y = batch_y.reshape((batch_size, num_classes))

        #sess.run(train_op, feed_dict={x: batch_x, y: batch_y})
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={X:batch_x,Y:batch_y})
            los=sess.run(loss_op,feed_dict={X:batch_x,Y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            preds = sess.run(prediction, feed_dict={X: batch_x})
            print("Prediction: ")
            print preds
            print("Actual: ")
            print batch_y
            print("__________________")
        iter=iter+1


('For iter ', 10)
('Accuracy ', 0.040000003)
('Loss ', 0.040000003)
Prediction: 
[[9.6926295e-12]]
Actual: 
[[0.2]]
__________________
('For iter ', 20)
('Accuracy ', 0.64000005)
('Loss ', 0.64000005)
Prediction: 
[[6.1444366e-13]]
Actual: 
[[0.8]]
__________________
('For iter ', 30)
('Accuracy ', 0.5625)
('Loss ', 0.5625)
Prediction: 
[[2.1413604e-13]]
Actual: 
[[0.75]]
__________________
('For iter ', 40)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.4163411e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 50)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.2011461e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 60)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.1278883e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 70)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.10301186e-13]]
Actual: 
[[0.55]]
__________________
('For iter ', 80)
('Accuracy ', 0.0625)
('Loss ', 0.0625)
Prediction: 
[[1.09587984e-13]]
Actual: 
[[0.25]]
________

('For iter ', 640)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.087557e-13]]
Actual: 
[[0.4]]
__________________
('For iter ', 650)
('Accuracy ', 0.0225)
('Loss ', 0.0225)
Prediction: 
[[1.0861535e-13]]
Actual: 
[[0.15]]
__________________
('For iter ', 660)
('Accuracy ', 0.010000001)
('Loss ', 0.010000001)
Prediction: 
[[1.085847e-13]]
Actual: 
[[0.1]]
__________________
('For iter ', 670)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0869908e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 680)
('Accuracy ', 0.5625)
('Loss ', 0.5625)
Prediction: 
[[1.09076864e-13]]
Actual: 
[[0.75]]
__________________
('For iter ', 690)
('Accuracy ', 0.80999994)
('Loss ', 0.80999994)
Prediction: 
[[1.08847013e-13]]
Actual: 
[[0.9]]
__________________
('For iter ', 700)
('Accuracy ', 0.64000005)
('Loss ', 0.64000005)
Prediction: 
[[1.088306e-13]]
Actual: 
[[0.8]]
__________________
('For iter ', 710)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.087333e-13]

('For iter ', 1290)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.0879491e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 1300)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.0915283e-13]]
Actual: 
[[0.4]]
__________________
('For iter ', 1310)
('Accuracy ', 0.48999998)
('Loss ', 0.48999998)
Prediction: 
[[1.0883684e-13]]
Actual: 
[[0.7]]
__________________
('For iter ', 1320)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.0857973e-13]]
Actual: 
[[0.55]]
__________________
('For iter ', 1330)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.0878806e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 1340)
('Accuracy ', 0.122499995)
('Loss ', 0.122499995)
Prediction: 
[[1.0879429e-13]]
Actual: 
[[0.35]]
__________________
('For iter ', 1350)
('Accuracy ', 0.0625)
('Loss ', 0.0625)
Prediction: 
[[1.09177614e-13]]
Actual: 
[[0.25]]
__________________
('For iter ', 1360)
('Accuracy ', 0.48999998)
('Loss ', 0.48999998)
Predicti

('For iter ', 1940)
('Accuracy ', 0.48999998)
('Loss ', 0.48999998)
Prediction: 
[[1.0908082e-13]]
Actual: 
[[0.7]]
__________________
('For iter ', 1950)
('Accuracy ', 0.040000003)
('Loss ', 0.040000003)
Prediction: 
[[1.091112e-13]]
Actual: 
[[0.2]]
__________________
('For iter ', 1960)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.0879615e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 1970)
('Accuracy ', 0.5625)
('Loss ', 0.5625)
Prediction: 
[[1.08741595e-13]]
Actual: 
[[0.75]]
__________________
('For iter ', 1980)
('Accuracy ', 0.5625)
('Loss ', 0.5625)
Prediction: 
[[1.0903215e-13]]
Actual: 
[[0.75]]
__________________
('For iter ', 1990)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0905232e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 2000)
('Accuracy ', 0.48999998)
('Loss ', 0.48999998)
Prediction: 
[[1.0882231e-13]]
Actual: 
[[0.7]]
__________________
('For iter ', 2010)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction

('For iter ', 2590)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0880467e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 2600)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0861598e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 2610)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.09137844e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 2620)
('Accuracy ', 0.5625)
('Loss ', 0.5625)
Prediction: 
[[1.0917406e-13]]
Actual: 
[[0.75]]
__________________
('For iter ', 2630)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.0918219e-13]]
Actual: 
[[0.4]]
__________________
('For iter ', 2640)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.0911203e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 2650)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.0872375e-13]]
Actual: 
[[0.55]]
__________________
('For iter ', 2660)
('Accuracy ', 0.0025000002)
('Loss ', 0.0025000002)
Prediction: 
[[1.08741595e-13]]
Actual: 
[[0

...

('For iter ', 7140)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.0914846e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 7150)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.0908622e-13]]
Actual: 
[[0.55]]
__________________
('For iter ', 7160)
('Accuracy ', 0.20249999)
('Loss ', 0.20249999)
Prediction: 
[[1.0870613e-13]]
Actual: 
[[0.45]]
__________________
('For iter ', 7170)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.0874532e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 7180)
('Accuracy ', 0.09)
('Loss ', 0.09)
Prediction: 
[[1.0883953e-13]]
Actual: 
[[0.3]]
__________________
('For iter ', 7190)
('Accuracy ', 0.80999994)
('Loss ', 0.80999994)
Prediction: 
[[1.0914097e-13]]
Actual: 
[[0.9]]
__________________
('For iter ', 7200)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0918698e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 7210)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.0874554e-13]]
Actua

('For iter ', 7790)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.08818363e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 7800)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.08709665e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 7810)
('Accuracy ', 0.0625)
('Loss ', 0.0625)
Prediction: 
[[1.0917199e-13]]
Actual: 
[[0.25]]
__________________
('For iter ', 7820)
('Accuracy ', 0.64000005)
('Loss ', 0.64000005)
Prediction: 
[[1.088468e-13]]
Actual: 
[[0.8]]
__________________
('For iter ', 7830)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.0861929e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 7840)
('Accuracy ', 0.7225)
('Loss ', 0.7225)
Prediction: 
[[1.0914534e-13]]
Actual: 
[[0.85]]
__________________
('For iter ', 7850)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0919176e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 7860)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.09187

('For iter ', 8440)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.09127856e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 8450)
('Accuracy ', 0.122499995)
('Loss ', 0.122499995)
Prediction: 
[[1.0874844e-13]]
Actual: 
[[0.35]]
__________________
('For iter ', 8460)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.088165e-13]]
Actual: 
[[0.4]]
__________________
('For iter ', 8470)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.08720446e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 8480)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.0920718e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 8490)
('Accuracy ', 0.010000001)
('Loss ', 0.010000001)
Prediction: 
[[1.08816906e-13]]
Actual: 
[[0.1]]
__________________
('For iter ', 8500)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.0861017e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 8510)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.0916512e-1

('For iter ', 9090)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.0924281e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 9100)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.09087896e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 9110)
('Accuracy ', 0.3025)
('Loss ', 0.3025)
Prediction: 
[[1.0877271e-13]]
Actual: 
[[0.55]]
__________________
('For iter ', 9120)
('Accuracy ', 0.010000001)
('Loss ', 0.010000001)
Prediction: 
[[1.08655354e-13]]
Actual: 
[[0.1]]
__________________
('For iter ', 9130)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.08885835e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 9140)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.0866654e-13]]
Actual: 
[[0.4]]
__________________
('For iter ', 9150)
('Accuracy ', 0.48999998)
('Loss ', 0.48999998)
Prediction: 
[[1.0888273e-13]]
Actual: 
[[0.7]]
__________________
('For iter ', 9160)
('Accuracy ', 0.48999998)
('Loss ', 0.48999998)
Prediction: 

('For iter ', 9740)
('Accuracy ', 0.040000003)
('Loss ', 0.040000003)
Prediction: 
[[1.08789314e-13]]
Actual: 
[[0.2]]
__________________
('For iter ', 9750)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[[1.0913638e-13]]
Actual: 
[[0.5]]
__________________
('For iter ', 9760)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.08663634e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 9770)
('Accuracy ', 0.040000003)
('Loss ', 0.040000003)
Prediction: 
[[1.0891222e-13]]
Actual: 
[[0.2]]
__________________
('For iter ', 9780)
('Accuracy ', 0.16000001)
('Loss ', 0.16000001)
Prediction: 
[[1.0889249e-13]]
Actual: 
[[0.4]]
__________________
('For iter ', 9790)
('Accuracy ', 0.36)
('Loss ', 0.36)
Prediction: 
[[1.09226145e-13]]
Actual: 
[[0.6]]
__________________
('For iter ', 9800)
('Accuracy ', 0.42249995)
('Loss ', 0.42249995)
Prediction: 
[[1.0893819e-13]]
Actual: 
[[0.65]]
__________________
('For iter ', 9810)
('Accuracy ', 0.25)
('Loss ', 0.25)
Prediction: 
[